In [ ]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from keras.models import load_model

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"lovemeifucan","key":"f2ffe865214ae3a79ba88ceea137adb8"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets list

ref                                                       title                                               size  lastUpdated          downloadCount  
--------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  
terenceshin/covid19s-impact-on-airport-traffic            COVID-19's Impact on Airport Traffic               106KB  2020-10-19 12:40:17           1295  
sootersaalu/amazon-top-50-bestselling-books-2009-2019     Amazon Top 50 Bestselling Books 2009 - 2019         15KB  2020-10-13 09:39:21           1262  
thomaskonstantin/highly-rated-children-books-and-stories  Highly Rated Children Books And Stories            106KB  2020-10-24 12:09:59            289  
tunguz/euro-parliament-proceedings-1996-2011              Euro Parliament Proceedings 1996 - 2011              1GB  2020-10-26 17:48:29             18  
rishidamarla/judicial-expenditures-across-all-50-states   Judicial Expenditures ac

In [ ]:
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

100% 2.29G/2.29G [00:30<00:00, 11.9MB/s]
100% 2.29G/2.29G [00:30<00:00, 79.7MB/s]


In [ ]:
!unzip chest-xray-pneumonia.zip

Streaming output truncated to the last 5000 lines.
  inflating: chest_xray/train/NORMAL/IM-0435-0001.jpeg  
  inflating: chest_xray/train/NORMAL/IM-0437-0001-0001.jpeg  
  inflating: chest_xray/train/NORMAL/IM-0437-0001-0002.jpeg  
  inflating: chest_xray/train/NORMAL/IM-0437-0001.jpeg  
  inflating: chest_xray/train/NORMAL/IM-0438-0001.jpeg  
  inflating: chest_xray/train/NORMAL/IM-0439-0001-0001.jpeg  
  inflating: chest_xray/train/NORMAL/IM-0439-0001-0002.jpeg  
  inflating: chest_xray/train/NORMAL/IM-0439-0001.jpeg  
  inflating: chest_xray/train/NORMAL/IM-0440-0001.jpeg  
  inflating: chest_xray/train/NORMAL/IM-0441-0001.jpeg  
  inflating: chest_xray/train/NORMAL/IM-0442-0001.jpeg  
  inflating: chest_xray/train/NORMAL/IM-0444-0001.jpeg  
  inflating: chest_xray/train/NORMAL/IM-0445-0001.jpeg  
  inflating: chest_xray/train/NORMAL/IM-0446-0001.jpeg  
  inflating: chest_xray/train/NORMAL/IM-0447-0001.jpeg  
  inflating: chest_xray/train/NORMAL/IM-0448-0001.jpeg  
  inflating: ches

In [ ]:
from keras.layers import Input, Lambda, Dense, Flatten,Conv2D,MaxPooling2D,Dropout,Flatten
from keras.models import Model,Sequential
#from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator,img_to_array,array_to_img
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [ ]:
IMAGE_SIZE = [224, 224]

train_path = 'Datasets/train'
valid_path = 'Datasets/test'

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('chest_xray/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('chest_xray/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')



Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [ ]:
model=Sequential()
model.add(Conv2D(64,kernel_size=(3,3),activation='relu',input_shape=[224,224,3]))
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2,activation='softmax'))
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit_generator(training_set,validation_data=test_set,epochs=5,steps_per_epoch=len(training_set),validation_steps=(len(test_set)))


Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/5
163/163 [==============================] - 152s 935ms/step - loss: 1.1577 - accuracy: 0.7977 - val_loss: 0.9017 - val_accuracy: 0.6442
Epoch 2/5
163/163 [==============================] - 153s 936ms/step - loss: 0.2951 - accuracy: 0.8777 - val_loss: 0.5272 - val_accuracy: 0.8013
Epoch 3/5
163/163 [==============================] - 153s 940ms/step - loss: 0.2454 - accuracy: 0.8990 - val_loss: 0.6448 - val_accuracy: 0.8173
Epoch 4/5
163/163 [==============================] - 149s 915ms/step - loss: 0.2493 - accuracy: 0.8995 - val_loss: 0.9048 - val_accuracy: 0.7548
Epoch 5/5
163/163 [==============================] - 147s 901ms/step - loss: 0.2293 - accuracy: 0.9043 - val_loss: 0.6386 - val_accuracy: 0.8109


In [ ]:
IMAGE_SIZE=[224,224]
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
# don't train existing weights
for layer in vgg.layers:
    layer.trainable = False
folders = glob('chest_xray/train/*')
x = Flatten()(vgg.output)
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model_1 = Model(inputs=vgg.input, outputs=prediction)

58892288/58889256 [==============================] - 0s 0us/step


In [ ]:
model_1.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [ ]:
model_1.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model_1.fit_generator(training_set,validation_data=test_set,epochs=10,steps_per_epoch=len(training_set),validation_steps=(len(test_set)))

Epoch 1/10
163/163 [==============================] - 127s 780ms/step - loss: 0.2178 - accuracy: 0.9179 - val_loss: 0.2698 - val_accuracy: 0.8990
Epoch 2/10
163/163 [==============================] - 124s 762ms/step - loss: 0.1155 - accuracy: 0.9540 - val_loss: 0.3789 - val_accuracy: 0.8958
Epoch 3/10
163/163 [==============================] - 124s 760ms/step - loss: 0.0951 - accuracy: 0.9649 - val_loss: 0.2576 - val_accuracy: 0.9199
Epoch 4/10
163/163 [==============================] - 124s 764ms/step - loss: 0.0959 - accuracy: 0.9617 - val_loss: 0.2675 - val_accuracy: 0.9151
Epoch 5/10
163/163 [==============================] - 124s 759ms/step - loss: 0.0823 - accuracy: 0.9701 - val_loss: 0.2358 - val_accuracy: 0.9151
Epoch 6/10
163/163 [==============================] - 125s 769ms/step - loss: 0.0739 - accuracy: 0.9737 - val_loss: 0.3836 - val_accuracy: 0.8766
Epoch 7/10
163/163 [==============================] - 124s 759ms/step - loss: 0.0598 - accuracy: 0.9791 - val_loss: 0.3164 -

In [ ]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
# don't train existing weights
for layer in vgg.layers:
    layer.trainable = True
folders = glob('chest_xray/train/*')
x = Flatten()(vgg.output)
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model_2 = Model(inputs=vgg.input, outputs=prediction)

In [ ]:
model_2.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [ ]:
model_2.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model_2.fit_generator(training_set,validation_data=test_set,epochs=20,steps_per_epoch=len(training_set),validation_steps=(len(test_set)))

Epoch 1/20
163/163 [==============================] - 157s 964ms/step - loss: 0.5592 - accuracy: 0.7573 - val_loss: 0.4354 - val_accuracy: 0.7821
Epoch 2/20
163/163 [==============================] - 157s 960ms/step - loss: 0.3207 - accuracy: 0.8683 - val_loss: 0.4070 - val_accuracy: 0.8189
Epoch 3/20
163/163 [==============================] - 155s 952ms/step - loss: 0.2173 - accuracy: 0.9097 - val_loss: 0.3334 - val_accuracy: 0.8686
Epoch 4/20
163/163 [==============================] - 155s 949ms/step - loss: 0.1965 - accuracy: 0.9183 - val_loss: 0.3137 - val_accuracy: 0.8750
Epoch 5/20
163/163 [==============================] - 151s 927ms/step - loss: 0.1927 - accuracy: 0.9220 - val_loss: 0.3076 - val_accuracy: 0.8782
Epoch 6/20
163/163 [==============================] - 150s 918ms/step - loss: 0.1865 - accuracy: 0.9262 - val_loss: 0.5476 - val_accuracy: 0.8061
Epoch 7/20
163/163 [==============================] - 146s 898ms/step - loss: 0.1732 - accuracy: 0.9314 - val_loss: 5.9124 -

In [ ]:
x=training_set.labels
len(x)

5216

In [ ]:
def preprocess_input(x):
  img=keras.applications.xception.preprocess_input(img_to_array(x))
  return array_to_img(x)

In [ ]:
k=0
for i in x:
  if(i==1):
    k=k+1

print(k)

3875


In [ ]:
IMAGE_SIZE = [299, 299]

train_path = 'Datasets/train'
valid_path = 'Datasets/test'

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

training_set = train_datagen.flow_from_directory('chest_xray/train',
                                                 target_size = (299, 299),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('chest_xray/test',
                                            target_size = (299, 299),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [ ]:
from keras.applications import InceptionResNetV2
model_3=InceptionResNetV2(include_top=False,weights="imagenet",input_shape=(299, 299, 3),pooling='max',classes=1000,classifier_activation=None)
folders = glob('chest_xray/train/*')
x = Flatten()(model_3.output)
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model_aditya = Model(inputs=model_3.input, outputs=prediction)
model_aditya.summary()

219062272/219055592 [==============================] - 6s 0us/step
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d_203 (Conv2D)             (None, 149, 149, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_203 (BatchN (None, 149, 149, 32) 96          conv2d_203[0][0]                 
__________________________________________________________________________________________________
activation_203 (Activation)     (None, 149, 149, 32) 0           batch_normalization_203[0][0]    
____________________

In [ ]:
model_aditya.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model_aditya.fit_generator(training_set,validation_data=test_set,epochs=20,steps_per_epoch=len(training_set),validation_steps=(len(test_set)))

Epoch 1/20
163/163 [==============================] - 172s 1s/step - loss: 0.4755 - accuracy: 0.9210 - val_loss: 0.6877 - val_accuracy: 0.7628
Epoch 2/20
163/163 [==============================] - 169s 1s/step - loss: 0.0945 - accuracy: 0.9676 - val_loss: 0.5279 - val_accuracy: 0.8365
Epoch 3/20
163/163 [==============================] - 169s 1s/step - loss: 0.0595 - accuracy: 0.9768 - val_loss: 0.2513 - val_accuracy: 0.9167
Epoch 4/20
163/163 [==============================] - 172s 1s/step - loss: 0.4755 - accuracy: 0.9210 - val_loss: 0.6877 - val_accuracy: 0.7628
Epoch 2/20
163/163 [==============================] - 169s 1s/step - loss: 0.0945 - accuracy: 0.9676 - val_loss: 0.5279 - val_accuracy: 0.8365
Epoch 3/20
163/163 [==============================] - 169s 1s/step - loss: 0.0595 - accuracy: 0.9768 - val_loss: 0.2513 - val_accuracy: 0.9167
Epoch 4/20
163/163 [==============================] - 169s 1s/step - loss: 0.0543 - accuracy: 0.9816 - val_loss: 0.2983 - val_accuracy: 0.9006

In [ ]:
model_4=InceptionResNetV2(include_top=False,weights="imagenet",input_shape=(299, 299, 3),pooling='max',classes=1000,classifier_activation=None)
folders = glob('chest_xray/train/*')
x = Flatten()(model_4.output)
prediction = Dense(len(folders), activation='softmax')(x)
for layer in model_4.layers:
    layer.trainable = False

model_shubha = Model(inputs=model_4.input, outputs=prediction)
model_shubha.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d_406 (Conv2D)             (None, 149, 149, 32) 864         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_406 (BatchN (None, 149, 149, 32) 96          conv2d_406[0][0]                 
__________________________________________________________________________________________________
activation_406 (Activation)     (None, 149, 149, 32) 0           batch_normalization_406[0][0]    
_______________________________________________________________________________________

In [ ]:
model_shubha.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
#model_shubha.fit_generator(training_set,validation_data=test_set,epochs=20,steps_per_epoch=len(training_set),validation_steps=(len(test_set)))
#this pretrained ANN on image net was giving poor accuracy than trainable,so no need to run this

In [ ]:
model_aditya.predict

array([[1.33607406e-02, 9.86639202e-01],
       [1.76101516e-04, 9.99823868e-01],
       [1.24719721e-04, 9.99875307e-01],
       [9.67462718e-01, 3.25372778e-02],
       [1.23905810e-03, 9.98760939e-01],
       [7.47605460e-03, 9.92523968e-01],
       [1.22255273e-01, 8.77744734e-01],
       [7.74944929e-05, 9.99922514e-01],
       [9.84401524e-01, 1.55984657e-02],
       [5.05080540e-03, 9.94949222e-01],
       [3.43125202e-02, 9.65687454e-01],
       [9.27192569e-01, 7.28074536e-02],
       [4.10051143e-04, 9.99589980e-01],
       [9.11128148e-03, 9.90888715e-01],
       [9.40698028e-01, 5.93019426e-02],
       [5.70643810e-04, 9.99429286e-01],
       [9.88365650e-01, 1.16343815e-02],
       [1.06929091e-03, 9.98930752e-01],
       [8.08803082e-01, 1.91196904e-01],
       [8.31504352e-04, 9.99168515e-01],
       [1.71183422e-03, 9.98288095e-01],
       [1.75571724e-04, 9.99824464e-01],
       [3.89331872e-05, 9.99961019e-01],
       [9.87488806e-01, 1.25111779e-02],
       [4.571767

In [ ]:
img=image.load_img('/content/chest_xray/chest_xray/test/PNEUMONIA/person100_bacteria_475.jpeg',target_size=[299,299])
x=image.img_to_array(img)
x=np.expand_dims(x, axis=0)
x=preprocess_input(x)
model_aditya=load_model('/content/sample_data/pneumonia.h5')
classes=model_aditya.predict(x)

In [ ]:
print(classes)

[[0.03234388 0.96765614]]


so our prediction is correct ,this person was having pneumonia


In [ ]:
from keras.applications import Xception
model_3=Xception(include_top=False,weights="imagenet",input_shape=(299, 299, 3),pooling='max',classes=1000,classifier_activation=None)
folders = glob('chest_xray/train/*')
x = Flatten()(model_3.output)
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model_aditya = Model(inputs=model_3.input, outputs=prediction)
model_aditya.summary()

83689472/83683744 [==============================] - 1s 0us/step
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
______________________

In [ ]:
model_aditya.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model_aditya.fit_generator(training_set,validation_data=test_set,epochs=20,steps_per_epoch=len(training_set),validation_steps=(len(test_set)))

Epoch 1/20
163/163 [==============================] - 239s 1s/step - loss: 0.1731 - accuracy: 0.9452 - val_loss: 0.5173 - val_accuracy: 0.8349
Epoch 2/20
163/163 [==============================] - 238s 1s/step - loss: 0.0856 - accuracy: 0.9699 - val_loss: 0.4892 - val_accuracy: 0.8590
Epoch 3/20
163/163 [==============================] - 241s 1s/step - loss: 0.0702 - accuracy: 0.9758 - val_loss: 0.8630 - val_accuracy: 0.7981
Epoch 4/20
163/163 [==============================] - 241s 1s/step - loss: 0.0545 - accuracy: 0.9820 - val_loss: 0.9263 - val_accuracy: 0.7420
Epoch 5/20
163/163 [==============================] - 242s 1s/step - loss: 0.0513 - accuracy: 0.9814 - val_loss: 0.7388 - val_accuracy: 0.8013
Epoch 6/20
163/163 [==============================] - 241s 1s/step - loss: 0.0413 - accuracy: 0.9847 - val_loss: 0.2827 - val_accuracy: 0.9263
Epoch 7/20
163/163 [==============================] - 240s 1s/step - loss: 0.0380 - accuracy: 0.9862 - val_loss: 0.3192 - val_accuracy: 0.9054

In [ ]:
model_aditya.save("pneumonia.h5")